In [1]:
# Importing necessary lebraries
import numpy as np
import pandas as pd
import os
import sys


In [2]:
sys.path.append(os.path.join(os.path.abspath('..')))
# Import modules
from src import data_loading as dl

In [3]:

fraud_df = dl.load_data("Fraud_Data.csv")
credit_df = dl.load_data("creditcard.csv")

In [6]:

sys.path.append(os.path.abspath("../"))


from scripts.model import prepare_data, split_data

from scripts.logger import logger 

 Feature and Target Separation for creditcard.csv

In [7]:

X_credit, y_credit = prepare_data(credit_df, 'Class')

INFO:fraud_detection_logger:Preparing data by separating features and target column: Class


2025-02-07 08:27:14,564 - INFO - Preparing data by separating features and target column: Class


Train-Test Split for creditcard.csv

In [8]:

X_train_credit, X_test_credit, y_train_credit, y_test_credit = split_data(X_credit, y_credit)

INFO:fraud_detection_logger:Splitting data into train and test sets


2025-02-07 08:27:19,800 - INFO - Splitting data into train and test sets


Feature and Target Separation for Fraud_Data.csv

In [9]:

X_fraud, y_fraud = prepare_data(fraud_df, 'class')

INFO:fraud_detection_logger:Preparing data by separating features and target column: class


2025-02-07 08:27:26,030 - INFO - Preparing data by separating features and target column: class


Train-Test Split for Fraud_Data.csv

In [10]:

X_train_fraud, X_test_fraud, y_train_fraud, y_test_fraud = split_data(X_fraud, y_fraud)

INFO:fraud_detection_logger:Splitting data into train and test sets


2025-02-07 08:27:28,741 - INFO - Splitting data into train and test sets
